# PyCity Schools Analysis

-As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

-As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).


-As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school.
  
---

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Calculate the total number of unique schools
school_count = len(school_data_complete["school_name"].unique())
school_count_total = school_data_complete["Student ID"].count()
school_count_total

39170

In [3]:
# Calculate the total number of students
student_count = school_data_complete["Student ID"].nunique()
student_count

39170

In [4]:
# Calculate the total budget
total_budget = school_data_complete["budget"].unique().sum()
total_budget

24649428

In [5]:
# Calculate the average (mean) math score
average_math_score = school_data_complete["math_score"].sum()/len(school_data_complete["math_score"])
average_math_score

78.98537145774827

In [6]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].sum()/len(school_data_complete["reading_score"])
average_reading_score

81.87784018381414

In [7]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [8]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete["Student ID"].nunique()
passing_reading_count= school_data_complete[(school_data_complete["reading_score"]>= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count)*100
passing_reading_percentage 

85.80546336482001

In [9]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [10]:
# Create a high-level snapshot of the district's key metrics in a DataFrame

district_summary = pd.DataFrame([{"Total Schools" : school_count,
                                "Total Students" : student_count,
                                "Total Budget" : total_budget,
                                "Average Math Score" : average_math_score,
                                "Average Reading Score" : average_reading_score,
                                "% Passing Math" : passing_math_percentage,
                                "% Passing Reading": passing_reading_percentage,
                                "% Overall Passing Rate" : overall_passing_rate}]) 

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [11]:
# Use the code provided to select all of the school types
school_types = school_data.set_index(["school_name"])["type"]
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [12]:
# Calculate the total student count per school
per_school_counts =  school_data_complete["student_name"].count()
per_school_counts

39170

In [13]:
# Calculate the total school budget and per capita spending per school
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts
per_school_budget
per_school_capita

/var/folders/df/cg719pr971j5qxlmf3lsdly00000gn/T/ipykernel_8358/464265912.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]


school_name
Bailey High School       79.778606
Cabrera High School      27.606740
Figueroa High School     48.108527
Ford High School         45.032321
Griffin High School      23.423538
Hernandez High School    77.151391
Holden High School        6.333597
Huang High School        48.778019
Johnson High School      79.005617
Pena High School         14.956804
Rodriguez High School    65.033521
Shelton High School      26.974726
Thomas High School       26.630840
Wilson High School       33.688384
Wright High School       26.790911
Name: budget, dtype: float64

In [14]:
# Calculate the average test scores per school
per_school_math = school_data_complete.groupby(["school_name"]).mean()["math_score"] 
per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"] 
per_school_math

/var/folders/df/cg719pr971j5qxlmf3lsdly00000gn/T/ipykernel_8358/464932101.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  per_school_math = school_data_complete.groupby(["school_name"]).mean()["math_score"]
/var/folders/df/cg719pr971j5qxlmf3lsdly00000gn/T/ipykernel_8358/464932101.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"]


school_name
Bailey High School       77.048432
Cabrera High School      83.061895
Figueroa High School     76.711767
Ford High School         77.102592
Griffin High School      83.351499
Hernandez High School    77.289752
Holden High School       83.803279
Huang High School        76.629414
Johnson High School      77.072464
Pena High School         83.839917
Rodriguez High School    76.842711
Shelton High School      83.359455
Thomas High School       83.418349
Wilson High School       83.274201
Wright High School       83.682222
Name: math_score, dtype: float64

In [15]:
per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"]
per_school_reading

/var/folders/df/cg719pr971j5qxlmf3lsdly00000gn/T/ipykernel_8358/1715023855.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"]


school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.725724
Thomas High School       83.848930
Wilson High School       83.989488
Wright High School       83.955000
Name: reading_score, dtype: float64

In [16]:
# Calculate the number of students per school with math scores of 70 or higher
school_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]
school_passing_reading
 

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [17]:
# Calculate the number of students per school with reading scores of 70 or higher
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] /per_school_counts * 100
per_school_passing_reading


school_name
Bailey High School       10.408476
Cabrera High School       4.603013
Figueroa High School      6.078632
Ford High School          5.545060
Griffin High School       3.640541
Hernandez High School     9.568547
Holden High School        1.049272
Huang High School         6.055655
Johnson High School       9.872351
Pena High School          2.356395
Rodriguez High School     8.189941
Shelton High School       4.309420
Thomas High School        4.061782
Wilson High School        5.626755
Wright High School        4.439622
Name: student_name, dtype: float64

In [18]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)]
passing_math_and_reading

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [19]:
passing_math_and_reading_per_school = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)].groupby("school_name")["student_name"].count()
total_students_per_school = school_data_complete.groupby("school_name")["student_name"].count()
passing_rates_per_school = (passing_math_and_reading_per_school / total_students_per_school) * 100
passing_rates_per_school

school_name
Bailey High School       54.642283
Cabrera High School      91.334769
Figueroa High School     53.204476
Ford High School         54.289887
Griffin High School      90.599455
Hernandez High School    53.527508
Holden High School       89.227166
Huang High School        53.513884
Johnson High School      53.539172
Pena High School         90.540541
Rodriguez High School    52.988247
Shelton High School      89.892107
Thomas High School       90.948012
Wilson High School       90.582567
Wright High School       90.333333
Name: student_name, dtype: float64

In [20]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
total_students_per_school = school_data_complete.groupby("school_name")["student_name"].count()
passing_math_and_reading_per_school = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)].groupby("school_name")["student_name"].count()
passing_rates_per_school = (passing_math_and_reading_per_school / total_students_per_school) * 100
per_school_summary = pd.DataFrame({
    "Total Students": total_students_per_school,
    "Passing Math and Reading": passing_math_and_reading_per_school,
    "Passing Rate": passing_rates_per_school
})

print(per_school_summary)


                       Total Students  Passing Math and Reading  Passing Rate
school_name                                                                  
Bailey High School               4976                      2719     54.642283
Cabrera High School              1858                      1697     91.334769
Figueroa High School             2949                      1569     53.204476
Ford High School                 2739                      1487     54.289887
Griffin High School              1468                      1330     90.599455
Hernandez High School            4635                      2481     53.527508
Holden High School                427                       381     89.227166
Huang High School                2917                      1561     53.513884
Johnson High School              4761                      2549     53.539172
Pena High School                  962                       871     90.540541
Rodriguez High School            3999                      2119 

## Highest-Performing Schools (by % Overall Passing)

In [21]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
per_school_summary["% Overall Passing"] = (per_school_summary["Passing Math and Reading"] / per_school_summary["Total Students"]) * 100
per_school_summary_sorted = per_school_summary.sort_values("% Overall Passing", ascending=False)
top_5_schools = per_school_summary_sorted.head(5)
top_5_schools



,Total Students,Passing Math and Reading,Passing Rate,% Overall Passing
school_name,,,,
Cabrera High School,1858,1697,91.334769,91.334769
Thomas High School,1635,1487,90.948012,90.948012
Griffin High School,1468,1330,90.599455,90.599455
Wilson High School,2283,2068,90.582567,90.582567
Pena High School,962,871,90.540541,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [22]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(["% Overall Passing"], ascending= True) 
bottom_schools.head(5)

,Total Students,Passing Math and Reading,Passing Rate,% Overall Passing
school_name,,,,
Rodriguez High School,3999,2119,52.988247,52.988247
Figueroa High School,2949,1569,53.204476,53.204476
Huang High School,2917,1561,53.513884,53.513884
Hernandez High School,4635,2481,53.527508,53.527508
Johnson High School,4761,2549,53.539172,53.539172


## Math Scores by Grade

In [23]:
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")] 
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")] 
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean() 
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean() 
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean() 
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()

ninth_grader_math_scores = ninth_graders_scores["math_score"] 
tenth_grader_math_scores = tenth_graders_scores["math_score"] 
eleventh_grader_math_scores = eleventh_graders_scores["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

math_scores_by_grade = pd.DataFrame({"9th": ninth_grader_math_scores, 
                                     "10th": tenth_grader_math_scores, 
                                     "11th": eleventh_grader_math_scores,
                                    "12th": twelfth_grader_math_scores})
# Minor data wrangling
math_scores_by_grade.index.name = None 
math_scores_by_grade

/var/folders/df/cg719pr971j5qxlmf3lsdly00000gn/T/ipykernel_8358/1172227206.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
/var/folders/df/cg719pr971j5qxlmf3lsdly00000gn/T/ipykernel_8358/1172227206.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
/var/folders/df/cg719pr971j5qxlmf3lsdly00000gn/T/ipykernel_8358/1172227206.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to Fa

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [24]:
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")] 
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")] 
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")] 
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean() 
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean() 
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean() 
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()

ninth_grader_reading_scores = ninth_graders_scores["reading_score"] 
tenth_grader_reading_scores = tenth_graders_scores["reading_score"] 
eleventh_grader_reading_scores = eleventh_graders_scores["reading_score"] 
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

reading_scores_by_grade = pd.DataFrame({"9th": ninth_grader_reading_scores, 
                                        "10th": tenth_grader_reading_scores, 
                                        "11th": eleventh_grader_reading_scores,
                                    "12th": twelfth_grader_reading_scores})

reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]] 
reading_scores_by_grade.index.name = None

reading_scores_by_grade

/var/folders/df/cg719pr971j5qxlmf3lsdly00000gn/T/ipykernel_8358/458626860.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
/var/folders/df/cg719pr971j5qxlmf3lsdly00000gn/T/ipykernel_8358/458626860.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
/var/folders/df/cg719pr971j5qxlmf3lsdly00000gn/T/ipykernel_8358/458626860.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [25]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [26]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()
school_spending_df

,Total Students,Passing Math and Reading,Passing Rate,% Overall Passing
school_name,,,,
Bailey High School,4976,2719,54.642283,54.642283
Cabrera High School,1858,1697,91.334769,91.334769
Figueroa High School,2949,1569,53.204476,53.204476
Ford High School,2739,1487,54.289887,54.289887
Griffin High School,1468,1330,90.599455,90.599455
Hernandez High School,4635,2481,53.527508,53.527508
Holden High School,427,381,89.227166,89.227166
Huang High School,2917,1561,53.513884,53.513884
Johnson High School,4761,2549,53.539172,53.539172


In [27]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, bins = spending_bins , labels = labels) 
school_spending_df

,Total Students,Passing Math and Reading,Passing Rate,% Overall Passing,Spending Ranges (Per Student)
school_name,,,,,
Bailey High School,4976,2719,54.642283,54.642283,<$585
Cabrera High School,1858,1697,91.334769,91.334769,<$585
Figueroa High School,2949,1569,53.204476,53.204476,<$585
Ford High School,2739,1487,54.289887,54.289887,<$585
Griffin High School,1468,1330,90.599455,90.599455,<$585
Hernandez High School,4635,2481,53.527508,53.527508,<$585
Holden High School,427,381,89.227166,89.227166,<$585
Huang High School,2917,1561,53.513884,53.513884,<$585
Johnson High School,4761,2549,53.539172,53.539172,<$585


In [28]:
# Calculate averages for the desired columns
average_total_students = per_school_summary['Total Students'].mean()
average_passing_math_reading = per_school_summary['Passing Math and Reading'].mean()
average_passing_rate = per_school_summary['Passing Rate'].mean()
average_overall_passing = per_school_summary['% Overall Passing'].mean()

# Print the calculated averages
print("Average Total Students:", average_total_students)
print("Average Passing Math and Reading:", average_passing_math_reading)
print("Average Passing Rate:", average_passing_rate)
print("Average Overall Passing:", average_overall_passing)


Average Total Students: 2611.3333333333335
Average Passing Math and Reading: 1701.8666666666666
Average Passing Rate: 73.27756047612858
Average Overall Passing: 73.27756047612858


## Scores by School Size

In [29]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [30]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"],bins=size_bins, labels =labels)
per_school_summary

,Total Students,Passing Math and Reading,Passing Rate,% Overall Passing,School Size
school_name,,,,,
Bailey High School,4976,2719,54.642283,54.642283,Large (2000-5000)
Cabrera High School,1858,1697,91.334769,91.334769,Medium (1000-2000)
Figueroa High School,2949,1569,53.204476,53.204476,Large (2000-5000)
Ford High School,2739,1487,54.289887,54.289887,Large (2000-5000)
Griffin High School,1468,1330,90.599455,90.599455,Medium (1000-2000)
Hernandez High School,4635,2481,53.527508,53.527508,Large (2000-5000)
Holden High School,427,381,89.227166,89.227166,Small (<1000)
Huang High School,2917,1561,53.513884,53.513884,Large (2000-5000)
Johnson High School,4761,2549,53.539172,53.539172,Large (2000-5000)


In [31]:
# Calculate averages for the desired columns. 
average_total_students = per_school_summary['Total Students'].mean()
average_passing_math_reading = per_school_summary['Passing Math and Reading'].mean()
average_passing_rate = per_school_summary['Passing Rate'].mean()
average_overall_passing = per_school_summary['% Overall Passing'].mean()

print("Average Total Students:", average_total_students)
print("Average Passing Math and Reading:", average_passing_math_reading)
print("Average Passing Rate:", average_passing_rate)
print("Average Overall Passing:", average_overall_passing)



Average Total Students: 2611.3333333333335
Average Passing Math and Reading: 1701.8666666666666
Average Passing Rate: 73.27756047612858
Average Overall Passing: 73.27756047612858


## Scores by School Type

In [32]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
# Create the "School Type" column in per_school_summary DataFrame
per_school_summary = per_school_summary.merge(school_data[["school_name", "type"]], on="school_name", how="left")
per_school_summary




,school_name,Total Students,Passing Math and Reading,Passing Rate,% Overall Passing,School Size,type
0,Bailey High School,4976,2719,54.642283,54.642283,Large (2000-5000),District
1,Cabrera High School,1858,1697,91.334769,91.334769,Medium (1000-2000),Charter
2,Figueroa High School,2949,1569,53.204476,53.204476,Large (2000-5000),District
3,Ford High School,2739,1487,54.289887,54.289887,Large (2000-5000),District
4,Griffin High School,1468,1330,90.599455,90.599455,Medium (1000-2000),Charter
5,Hernandez High School,4635,2481,53.527508,53.527508,Large (2000-5000),District
6,Holden High School,427,381,89.227166,89.227166,Small (<1000),Charter
7,Huang High School,2917,1561,53.513884,53.513884,Large (2000-5000),District
8,Johnson High School,4761,2549,53.539172,53.539172,Large (2000-5000),District
9,Pena High School,962,871,90.540541,90.540541,Small (<1000),Charter
